In [28]:
from torchvision import models, transforms
import torch
from PIL import Image
import numpy as np
import os

In [29]:
train_dir = '../../train'
gallery_dir = '../../test-2/gallery'
query_dir = '../../test-2/query'

In [30]:
device = torch.device("mps" if torch.mps.is_available() else "cpu")

# Load EfficientNetB0 and remove classifier
model = models.efficientnet_b0(pretrained=True)
model.classifier = torch.nn.Identity()
model.eval().to(device)

/Users/anthony/Desktop/UniTrento/Courses/Second Semester/I2ML/I2ML_ImageClassifier/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/anthony/Desktop/UniTrento/Courses/Second Semester/I2ML/I2ML_ImageClassifier/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

# Data Prep

In [31]:
# Preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [32]:
def extract_feature(img_path):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(x).squeeze().cpu().numpy()
    return embedding

In [33]:
def build_feature_index(folder_path):
    features = []
    filenames = []
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.jpg', '.png', '.jpeg')):
            path = os.path.join(folder_path, fname)
            feat = extract_feature(path)
            features.append(feat)
            filenames.append(fname)
    return np.array(features), filenames

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k(query_feature, gallery_features, gallery_filenames, k=5):
    sims = cosine_similarity(query_feature.reshape(1, -1), gallery_features).flatten()
    top_k_idx = sims.argsort()[-k:][::-1]
    return [(gallery_filenames[i], sims[i]) for i in top_k_idx]

In [35]:
submission = dict()

In [36]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
def create_dict_submission(gallery_embeddings, query_embeddings, similarity_matrix, query_paths, gallery_paths, k: int=5):
    submission = dict()
    
    gallery_embeddings = np.vstack(gallery_embeddings)
    query_embeddings = np.vstack(query_embeddings)

    top_k = k  # You can change this to any value (e.g., 1, 3, 10)

    # Compute top-k most similar gallery indices for each query
    topk_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]

    # Display results
    for i, indices in enumerate(topk_indices):
        print(f"\nQuery image: {query_paths[i]}")
        print("Top {} retrieved gallery images:".format(top_k))
        # finds index of the rightmost slash so that strings can be subset to only include the name
        # of the file and not the entire path leading to it
        idx_last_slash = str(query_paths[i]).rfind("/")
        submission[str(query_paths[i][idx_last_slash+1:])] = list()
        for rank, gallery_idx in enumerate(indices):
            print(f"  Rank {rank+1}: {gallery_paths[gallery_idx]}")
            # finds index of the rightmost slash so that strings can be subset to only include the name
            # of the file and not the entire path leading to it
            idx_last_slash_res = str(gallery_paths[gallery_idx]).rfind("/")
            submission[str(query_paths[i][idx_last_slash+1:])].append(gallery_paths[gallery_idx][idx_last_slash_res+1:])

    return submission

In [37]:
for q_feat, q_name in zip(query_features, query_filenames):
    print(f"\nQuery: {q_name}")
    top_k = retrieve_top_k(q_feat, gallery_features, gallery_filenames, k=10)
    for fname, score in top_k:
        print(f"  → {fname} (similarity: {score:.4f})")
        if q_name not in submission:
            submission[q_name] = []
        submission[q_name].append(fname)


Query: 4f55487a739b4d1da78c3383d7b37ea2.jpg
  → e2bc5521d7374eeeb233df03c814fa2a.jpg (similarity: 0.6561)
  → 2bd9978849fc4561ab7f322e3bfbbd83.jpg (similarity: 0.6520)
  → 60deaa5529ae4f899d34f4accbb11661.jpg (similarity: 0.6248)
  → a75d79ce216f4c838eef372406848f14.jpg (similarity: 0.6240)
  → 049b584b3af9408493b432cfd9cba577.jpg (similarity: 0.6176)
  → 35e8624e2c31435ca232d9c2ab6a963c.jpg (similarity: 0.6141)
  → 826698e69ad843ddafe18fadeaae1091.jpg (similarity: 0.6065)
  → 225e33d1f0764943a6e88f880da423cc.jpg (similarity: 0.5960)
  → 8b11436b42864cfcaba961db4e6dfcd9.jpg (similarity: 0.5939)
  → 55987960e55541a983778be2751145e0.jpg (similarity: 0.5875)

Query: 06b470c625054ec4ac91d15a6808f15f.jpg
  → 50a45e63e946442fa3a0b6efc8a5a73a.jpg (similarity: 0.5058)
  → 57439bfcfb724b51a79380d1a7ddacb2.jpg (similarity: 0.4801)
  → 5e0b485b44484a75bdb79651469de5b2.jpg (similarity: 0.4742)
  → 9f2e65949b644b859d3383bac0028f86.jpg (similarity: 0.4490)
  → 1547328594ce4bc9b2f4842a69e03292.jpg (

In [39]:
len(submission['00502e372bc846c2b8d52780e5387587.jpg'])

10

# Submit

In [40]:
from utils.submit import submit

In [41]:
submit(submission, groupname='Overfit & Underpaid', url='http://tatooine.disi.unitn.it:3001/retrieval/')

accuracy is 33.74570446735395
